In [ ]:
from pathlib import Path

import cv2
import numpy as np
import pandas as pd

from window.postprocess.match_truths import match_yolo_to_truths
from window.utils.drawing import draw_results_on_image
from window.utils.tiling import split_locations_array
from window.utils.truths import windows_truth

In [ ]:
file_loc = '/home/cmf21/out_dets.csv'
boxes_in = pd.read_csv(file_loc)
boxes_in

In [ ]:
file_locs = boxes_in.file_loc.to_list()

def split_filename(str_in):
    file_nm = str_in[:-4]
    file_splt = file_nm.split('_')
    file_out = file_splt[0] + '_' + file_splt[1] + '.jpg'
    tile_out = file_splt[2]
    return file_out, tile_out

filenames = [split_filename(fl)[0] for fl in file_locs]
tiles = [split_filename(fl)[1] for fl in file_locs]


In [ ]:
boxes_in['image_names'] = filenames
boxes_in['tiles'] = tiles

In [ ]:
boxes_fil = boxes_in[boxes_in.prob > 0.5]
boxes_fil.shape

In [ ]:
def windows_to_whole_im(df_in):
    tile_w = 1856
    tile_h = 1256
    img_w = 7360
    img_h = 4912
    tile_vals = split_locations_array()
    xmin = df_in.xmin.tolist()
    xmax = df_in.xmax.tolist()
    ymin = df_in.ymin.tolist()
    ymax = df_in.ymax.tolist()
    tileord = df_in.tiles.astype(int).tolist()
    tilerowst = tile_vals[tileord, 0]
    tilecolst = tile_vals[tileord, 1]
    xmin = np.add(xmin, tilecolst)
    xmax = np.add(xmax, tilecolst)
    ymin = np.add(ymin, tilerowst)
    ymax = np.add(ymax, tilerowst)
    df_in.loc[:, 'xmn'] = np.array(xmin, dtype=np.int)
    df_in.loc[:, 'xmx'] = np.array(xmax, dtype=np.int)
    df_in.loc[:, 'ymn'] = np.array(ymin, dtype=np.int)
    df_in.loc[:, 'ymx'] = np.array(ymax, dtype=np.int)
    return df_in


windows_whl_im = windows_to_whole_im(boxes_fil)
windows_whl_im['conf'] = windows_whl_im['prob']


In [ ]:
truth_file = "/data/old_home_dir/ChrissyF/GFRC/yolo_valid_GFRC_bboxes.csv"
# Process truth files
truths = pd.read_csv(truth_file)
truths.loc[:, 'filename'] = [strin.replace('/', '_') for strin in truths.file_loc]

truths

In [ ]:
valid_whole_image_dir = "/data/old_home_dir/ChrissyF/GFRC/Valid/whole_images_all/"
# get all image files
image_files = list(Path(valid_whole_image_dir).glob("*.jpg"))
image_files = [img.name for img in image_files]

# create empty list to store results
results_all_ims = pd.DataFrame(columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'confmat', 'tru_box', 'filename'])

#fl = image_files[0]

In [ ]:
for fl in image_files:

    truths_im = truths[truths.filename == fl]
    truths_im_pixels = windows_truth(truths_im)

    # Per image
    print(fl)
    whole_im = cv2.imread(valid_whole_image_dir + fl)
    whole_im = cv2.cvtColor(whole_im, cv2.COLOR_BGR2RGB)

    windows_fil = windows_whl_im[windows_whl_im.image_names == fl]

    # calculate results
    iou_threshold = 0.15
    nms_threshold = 0.1
    results_per_im = match_yolo_to_truths(windows_fil, truths_im_pixels, iou_threshold, 0.1)
    results_per_im['filename'] = fl

    results_all_ims = pd.concat((results_all_ims, results_per_im), axis=0, sort=False)

    # draw results on image
    #image_out = draw_results_on_image(whole_im, results_per_im)

In [ ]:
tru_ims = np.unique(truths.filename).tolist()
ims_cv = np.unique(windows_whl_im.image_names).tolist()

match_tru = []
for tru in tru_ims:
    match_tru.append(tru in ims_cv)


In [ ]:
ims_cv_arr = np.array(ims_cv)
np.argmax(ims_cv_arr == tru_ims[0])

In [ ]:
print(f'TP: {np.sum(results_all_ims.confmat=="TP")}, FP: {np.sum(results_all_ims.confmat=="FP")}, FN: {np.sum(results_all_ims.confmat=="FN")}')
print("number of images:", len(image_files))

In [ ]:
im1 = 'Z247_Img14318.jpg'
im2 = 'Z124_Img13083.jpg'
im3 = 'Z138_Img02887.jpg'

np.sum(image_files == im3)


In [ ]:
fl = image_files[7]

truths_im = truths[truths.filename == fl]
truths_im_pixels = windows_truth(truths_im)

# Per image
print(fl)
whole_im = cv2.imread(valid_whole_image_dir + fl)
whole_im = cv2.cvtColor(whole_im, cv2.COLOR_BGR2RGB)

windows_fil = windows_whl_im[windows_whl_im.image_names == fl]

# calculate results
iou_threshold = 0.15
nms_threshold = 0.1
results_per_im = match_yolo_to_truths(windows_fil, truths_im_pixels, iou_threshold, 0.1)
results_per_im['filename'] = fl

results_all_ims = pd.concat((results_all_ims, results_per_im), axis=0, sort=False)

# draw results on image
image_out = draw_results_on_image(whole_im, results_per_im)
image_out = cv2.cvtColor(image_out, cv2.COLOR_BGR2RGB)
cv2.imwrite('hog_ex_', image_out)

In [ ]:
truths_im

In [ ]:
from PIL import Image

Image.fromarray(image_out)